In [ ]:

'Hello World!'

'Hello World!'

In [19]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

2.9.0+cu126
True
1
0
Tesla T4


In [20]:
import os
import math
import random
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"

In [21]:
class ImageDataset(Dataset):
    def __init__(self, root, size=64):
        self.files = glob(os.path.join(root, "**/*,jpg"), recursive=True)
        self.transform = transforms.Compose([
            transforms.Resize(size),
            transforms.CenterCrop(size),
            transforms.ToTensor(),
            transforms.Normalize([0.5]*3, [0.5]*3)
        ])
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        img = Image.open(self.files[idx]).convert("RGB")
        return self.transform(img)

In [22]:
class MappingNetwork(nn.Module):
    def __init__(self, z_dim=512, w_dim=512):
        super().__init__()
        layers = []
        for _ in range(8):
            layers.append(nn.Linear(w_dim, w_dim))
            layers.append(nn.LeakyReLU())
        self.mapping = nn.Sequential(*layers)
    
    def forward(self, z):
        z = z / z.norm(dim=1, keepdim=True)
        return self.mapping(z)

In [23]:
class StyleConv(nn.Module):
    def __init__(self, in_ch, out_ch, w_dim):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(out_ch, in_ch, 3, 3))
        self.style = nn.Linear(w_dim, in_ch)
        self.noise_strength = nn.Parameter(torch.zeros(1))
        self.bias = nn.Parameter(torch.zeros(out_ch))
    
    def forward(self, x, w, noise):
        b, c, h, w_ = x.shape
        style = self.style(w).view(b, 1, c, 1, 1)
        weight = self.weight.unsqueeze(0) * style
        weight = weight.view(-1, c, 3, 3)

        x = x.view(1, -1, h, w_)
        x= F.conv2d(x, weight, padding=1, groups=b)
        x = x.view(b, -1, h, w_)

        x = x + self.noise_strength * noise
        return x + self.bias.view(1, -1, 1, 1)

In [24]:
class Generator(nn.Module):
    def __init__(self, z_dim=512, w_dim=512):
        super().__init__()
        self.mapping = MappingNetwork(z_dim, w_dim)

        self.const = nn.Parameter(torch.randn(1, 512, 4, 4))

        self.layers = nn.ModuleList([
            StyleConv(512, 512, w_dim),
            StyleConv(512, 256, w_dim),
            StyleConv(256, 128, w_dim),
            StyleConv(128, 64, w_dim),
          ])
        
        self.to_rgb = nn.Conv2d(64, 3, 1)

    def forward(self, z):
        w = self.mapping(z)
        x = self.const.repeat(z.size(0), 1, 1, 1)

        for layer in self.layers:
            noise = torch.randn_like(x[:, :1])
            x = F.interpolate(x, scale_factor=2, mode='bilinear')
            x = layer(x, w, noise)
            x = F.leaky_relu(x, 0.2)

        return torch.tanh(self.to_rgb(x))

In [25]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        def block(in_c, out_c):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, 4, 2, 1),
                nn.LeakyReLU(0.2)
            )
        
        self.net = nn.Sequential(
            block(3, 64),
            block(64, 128),
            block(128, 256),
            block(256, 512),
            nn.Flatten(),
            nn.Linear(512*4*4, 1)
        )

    def forward(self, x):
        return self.net(x)


In [26]:
def Gradient_Penalty(D, real, fake):
    alpha = torch.rand(real.size(0), 1, 1, 1).to(device)
    interp = (alpha * real + (1 - alpha) * fake).requires_grad_(True)
    out = D(interp)

    grads = torch.autograd.grad(
        outputs=out,
        inputs=interp,
        grad_outputs=torch.ones_like(out),
        create_graph=True,
    )[0]

    return ((grads.norm(2, dim=1) - 1) ** 2).mean()

In [27]:
def train():
    path = r"C:\Users\TEJAS KUMAR V URS\Desktop\MSc Data Science\Projects\Animal Gen AI\Dataset"
    dataset = ImageDataset(path)
    loader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=2)

    G = Generator().to(device)
    D = Discriminator().to(device)

    g_opt = torch.optim.Adam(G.parameters(), lr=1e-4, betas=(0.0, 0.99))
    d_opt = torch.optim.Adam(D.parameters(), lr=1e-4, betas=(0.0, 0.99))

    for epoch in range(50):
        for i, real in enumerate(loader):
            real = real.to(device)
            z = torch.randn(real.size(0), 512).to(device)
            fake = G(z)

            d_loss = D(fake).mean() - D(real).mean()
            gp = Gradient_Penalty(D, real, fake)
            d_total = d_loss + 10 * gp

            d_opt.zero_grad()
            d_total.backward()
            d_opt.step()

            if i % 5 == 0:
                g_loss = -D(G(z)).mean()
                g_opt.zero_grad()
                g_loss.backward()
                g_opt.step()

        with torch.no_grad():
            samples = G(torch.randn(16, 512).to(device))
            save_image(samples, f"samples/epoch_{epoch}.png", normalize=True)

        print(f"Epoch {epoch} | D: {d_total.item():.3f} | G: {g_loss.item():.3f}")

In [14]:
if __name__ == "--main__":
    train()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\TEJAS KUMAR V URS\\Desktop\\MSc Data Science\\Projects\\Animal Gen AI\\Dataset'